# Jupyter notebook based on ImageD11 to process 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 27/02/2024

Now we have good experimental parameters, we can index more grains!

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import os, glob, pprint
import numpy as np
import h5py
from tqdm.notebook import tqdm

import matplotlib
%matplotlib widget
from matplotlib import pyplot as plt

# import utils
from ImageD11.nbGui import nb_utils as utils

import ImageD11.grain
import ImageD11.indexing
import ImageD11.columnfile
from ImageD11.sinograms import properties, dataset

from ImageD11.blobcorrector import eiger_spatial
from ImageD11.peakselect import select_ring_peaks_by_intensity

In [ ]:
# desination of H5 files

dset_path = '/data/visitor/ihma439/id11/20231211/PROCESSED_DATA/James/20240724/FeAu_0p5_tR/FeAu_0p5_tR_ff1/FeAu_0p5_tR_ff1_dataset.h5'

In [ ]:
# load the dataset from file

ds = ImageD11.sinograms.dataset.load(dset_path)

sample = ds.sample
dataset = ds.dset
print(ds)
print(ds.shape)

In [ ]:
# load 3d columnfile from disk

cf_3d = ds.get_cf_3d_from_disk()

cf_3d.parameters.loadparameters(ds.parfile)
cf_3d.updateGeometry()
cf_3d_path = f'{sample}_{dataset}_3d_peaks.flt'
cf_3d.writefile(cf_3d_path)

In [ ]:
# now we can define a unit cell from our parameters

ucell = ImageD11.unitcell.unitcell_from_parameters(cf_3d.parameters)
ucell.makerings(cf_3d.ds.max())

In [ ]:
# plot the 3D peaks (fewer of them) as a cake (two-theta vs eta)
# if the parameters in the par file are good, these should look like straight lines

fig, ax = plt.subplots()

ax.scatter(cf_3d.ds, cf_3d.eta, s=1)

ax.set_xlabel("D-star")
ax.set_ylabel("eta")

plt.show()

In [ ]:
# here we are filtering our peaks (cf_3d) to select only the strongest ones for indexing purposes only!
# dsmax is being set to limit rings given to the indexer - 6-8 rings is normally good

# USER: modify the "frac" parameter below and re-run the cell until the orange dot sits nicely on the "elbow" of the blue line
# this indicates the fractional intensity cutoff we will select
# if the blue line does not look elbow-shaped in the logscale plot, try changing the "doplot" parameter (the y scale of the logscale plot) until it does


cf_strong_frac = 0.9837
cf_strong_dsmax = 1.01
cf_strong_dstol = 0.01

cf_strong = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_frac, dsmax=cf_strong_dsmax, doplot=0.8, dstol=cf_strong_dstol)
print(f"Got {cf_strong.nrows} strong peaks for indexing")
cf_strong_path = f'{sample}_{dataset}_3d_peaks_strong.flt'
cf_strong.writefile(cf_strong_path)

In [ ]:
# we will also export some additional strong peaks across all rings
# this will be useful for grain refinement later (using makemap)

cf_strong_allrings_frac = cf_strong_frac
cf_strong_allrings_dstol = cf_strong_dstol

cf_strong_allrings = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_allrings_frac, dsmax=cf_3d.ds.max(), doplot=0.8, dstol=cf_strong_allrings_dstol)
print(f"Got {cf_strong_allrings.nrows} strong peaks for makemap")
cf_strong_allrings_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt'
cf_strong_allrings.writefile(cf_strong_allrings_path)

In [ ]:
# now we can take a look at the intensities of the remaining peaks

fig, ax = plt.subplots(figsize=(16, 9), constrained_layout=True)

ax.plot( ucell.ringds, [1e4,]*len(ucell.ringds), '|', ms=90, c="red")

ax.plot(cf_3d.ds, cf_3d.sum_intensity,',', label='cf_3d')
ax.plot(cf_strong.ds, cf_strong.sum_intensity,',', label='cf_strong')
ax.semilogy()

ax.set_xlabel("Dstar")
ax.set_ylabel("Intensity")
ax.legend()

plt.show()

In [ ]:
# specify our ImageD11 indexer with these peaks

indexer = ImageD11.indexing.indexer_from_colfile(cf_strong)

print(f"Indexing {cf_strong.nrows} peaks")

# USER: set a tolerance in d-space (for assigning peaks to powder rings)

indexer_ds_tol = 0.05
indexer.ds_tol = indexer_ds_tol

# change the log level so we can see what the ring assigments look like

ImageD11.indexing.loglevel = 1

# assign peaks to powder rings

indexer.assigntorings()

# change log level back again

ImageD11.indexing.loglevel = 3

In [ ]:
# let's plot the assigned peaks

fig, ax = plt.subplots()

# indexer.ra is the ring assignments

ax.scatter(cf_strong.ds, cf_strong.eta, c=indexer.ra, cmap='tab20', s=1)
ax.plot( ucell.ringds, [0,]*len(ucell.ringds), '|', ms=90, c="red")
ax.set_xlabel("d-star")
ax.set_ylabel("eta")
ax.set_xlim(cf_strong.ds.min()-0.05, cf_strong.ds.max()+0.05)

plt.show()

In [ ]:
# now we are indexing!
# we have to choose which rings we want to generate orientations on
# generally we want two or three low-multiplicity rings that are isolated from other phases
# take a look at the ring assignment output from a few cells above, and choose two or three
rings_for_gen = [0, 1]

# now we want to decide which rings to score our found orientations against
# generally we can just exclude dodgy rings (close to other phases, only a few peaks in etc)
rings_for_scoring = [0, 1, 2, 3]

# the sequence of hkl tolerances the indexer will iterate through
hkl_tols_seq = [0.01, 0.02, 0.03, 0.04]
# the sequence of minpks fractions the indexer will iterate through
fracs = [0.9, 0.75]
# the tolerance in g-vector angle
cosine_tol = np.cos(np.radians(90 - 0.25))
# the max number of UBIs we can find per pair of rings
max_grains = 1000

grains, indexer = utils.do_index(cf=cf_strong,
                                dstol=indexer.ds_tol,
                                forgen=rings_for_gen,
                                foridx=rings_for_scoring,
                                hkl_tols=hkl_tols_seq,
                                fracs=fracs,
                                cosine_tol=cosine_tol,
                                max_grains=max_grains
)

In [ ]:
# create grain objects
grains = [ImageD11.grain.grain(ubi, translation=np.array([0., 0., 0.])) for ubi in indexer.ubis]

# set grain GIDs (useful if we ever delete a grain)
for i, g in enumerate(grains):
    g.gid = i

In [ ]:
# let's look at the pole figures
# for this we need to set a reference unit cell for the grains

cf_pars = cf_strong.parameters.get_parameters()
spacegroup = 229  # spacegroup for BCC iron
cf_pars["cell_lattice_[P,A,B,C,I,F,R]"] = spacegroup

ref_ucell = ImageD11.unitcell.unitcell_from_parameters(cf_pars)

for g in grains:
    g.ref_unitcell = ref_ucell

In [ ]:
utils.plot_all_ipfs(grains)

In [ ]:
tmp_ubi_path = f'{sample}_{dataset}_grains.ubi'
tmp_map_path = f'{sample}_{dataset}_grains.map'

new_flt_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt.new'  # flt file containing assignments from makemap
unindexed_flt_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt.unindexed'  # remaining unassigned peaks from makemap

In [ ]:
ImageD11.grain.write_grain_file(tmp_ubi_path, grains)

In [ ]:
omegas_sorted = np.sort(ds.omega)[0]
omega_step = np.round(np.diff(omegas_sorted).mean(), 3)
omega_slop = omega_step/2

makemap_hkl_tol_seq = [0.05, 0.025, 0.01]

In [ ]:
symmetry = "cubic"

for inc, makemap_tol in enumerate(makemap_hkl_tol_seq):
    print(f"Running makemap {inc+1}/{len(makemap_hkl_tol_seq)}")
    if inc == 0:  # ubi into map
        makemap_output = !makemap.py -p {ds.parfile} -u {tmp_ubi_path} -U {tmp_map_path} -f {cf_strong_allrings_path} -F {unindexed_flt_path} -s {symmetry} -t {makemap_hkl_tol_seq[inc]} --omega_slop={omega_slop} --no_sort
    else:  # map into map
        makemap_output = !makemap.py -p {ds.parfile} -u {tmp_map_path} -U {tmp_map_path} -f {cf_strong_allrings_path} -F {unindexed_flt_path} -s {symmetry} -t {makemap_hkl_tol_seq[inc]} --omega_slop={omega_slop} --no_sort

In [ ]:
# re-import our refined grains from the makemap procedure

grains2 = ImageD11.grain.read_grain_file(tmp_map_path)

In [ ]:
# remove grains with no peaks

grains2 = [grain for grain in grains2 if "no peaks" not in grain.intensity_info]

In [ ]:
centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
xx = [grain.translation[0] for grain in grains2]
yy = [grain.translation[1] for grain in grains2]
zz = [grain.translation[2] for grain in grains2]
# col = [utils.grain_to_rgb(grain) for grain in grains2]  # IPF-Z colour instead
col = [float(grain.npks) for grain in grains2]
sizes = [0.01*(float(grain.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))) for grain in grains2]
if centre_plot:
    scatterplot = ax.scatter(xx-np.mean(xx), yy-np.mean(yy), zz, c=col, s=sizes)
else:
    scatterplot = ax.scatter(xx, yy, zz, c=col, s=sizes)
ax.set_xlim(-200,200)
ax.set_ylim(-200,200)
ax.set_zlim(-200,200)
plt.colorbar(scatterplot)
ax.set_title("Grains coloured by n peaks")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist([float(grain.npks) for grain in grains2], bins=30)
plt.show()

In [ ]:
# find the spike
absolute_minpks = 37

In [ ]:
# filter out grains with fewer than 15 peaks
grains_filtered = [grain for grain in grains2 if float(grain.npks) > absolute_minpks]

In [ ]:
centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
xx = [grain.translation[0] for grain in grains_filtered]
yy = [grain.translation[1] for grain in grains_filtered]
zz = [grain.translation[2] for grain in grains_filtered]
# col = [utils.grain_to_rgb(grain) for grain in grains_filtered]  # IPF-Z colour instead
col = [float(grain.npks) for grain in grains_filtered]
sizes = [0.01*(float(grain.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))) for grain in grains_filtered]
if centre_plot:
    scatterplot = ax.scatter(xx-np.mean(xx), yy-np.mean(yy), zz, c=col, s=sizes)
else:
    scatterplot = ax.scatter(xx, yy, zz, c=col, s=sizes)
ax.set_xlim(-200,200)
ax.set_ylim(-200,200)
ax.set_zlim(-200,200)
plt.colorbar(scatterplot)
ax.set_title("Grains coloured by n peaks")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

In [ ]:
# we now have our trustworthy grains
# we should run makemap again to regenerate our peak <-> grain assigments

map_path = f'{sample}_{dataset}_grains_filtered.map'
final_unindexed_flt_path = f'{sample}_{dataset}_3d_peaks.flt.unindexed'
final_new_flt_path = f'{sample}_{dataset}_3d_peaks.flt.new'

# write filtered grains to disk
ImageD11.grain.write_grain_file(map_path, grains_filtered)

# run makemap on filtered grains with all 3D peals
makemap_output = !makemap.py -p {ds.parfile} -u {map_path} -U {map_path} -f {cf_3d_path} -F {final_unindexed_flt_path} -s {symmetry} -t {makemap_hkl_tol_seq[-1]} --omega_slop={omega_slop} --no_sort

# import makemap output columnfile with peak assignments
cf_3d = ImageD11.columnfile.columnfile(final_new_flt_path)

# write 3D columnfile to disk
ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile, name='peaks')

# re-import filtered grains with new peak statistics
grains_filtered = ImageD11.grain.read_grain_file(map_path)

In [ ]:
# save grain data

ds.save_grains_to_disk(grains_filtered)

In [ ]:
centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
xx = [grain.translation[0] for grain in grains_filtered]
yy = [grain.translation[1] for grain in grains_filtered]
zz = [grain.translation[2] for grain in grains_filtered]
# col = [utils.grain_to_rgb(grain) for grain in grains_filtered]  # IPF-Z colour instead
col = [float(grain.npks) for grain in grains_filtered]
sizes = [0.01*(float(grain.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))) for grain in grains_filtered]
if centre_plot:
    scatterplot = ax.scatter(xx-np.mean(xx), yy-np.mean(yy), zz, c=col, s=sizes)
else:
    scatterplot = ax.scatter(xx, yy, zz, c=col, s=sizes)
ax.set_xlim(-200,200)
ax.set_ylim(-200,200)
ax.set_zlim(-200,200)
plt.colorbar(scatterplot)
ax.set_title("Grains coloured by n peaks")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

In [ ]:
# cleaning up

for path in [
    cf_3d_path,
    cf_strong_path,
    cf_strong_allrings_path,
    tmp_ubi_path,
    tmp_map_path,
    new_flt_path,
    unindexed_flt_path,
    map_path,
    final_unindexed_flt_path,
    final_new_flt_path
]:
    if os.path.exists(path):
        os.remove(path)

In [ ]:
# change to 0 to allow all cells to be run automatically
if 1:
    raise ValueError("Hello!")

In [ ]:
# Now that we're happy with our indexing parameters, we can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip in skips_dict

skips_dict = {
    "FeAu_0p5_tR": []
}

dset_prefix = "ff"

sample_list = ["FeAu_0p5_tR"]
    
samples_dict = utils.find_datasets_to_process(rawdata_path, skips_dict, dset_prefix, sample_list)


for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        print("Importing DataSet object")
        dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
        ds = ImageD11.sinograms.dataset.load(dset_path)
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        
        if os.path.exists(ds.grainsfile):
            print(f"Found existing grains file for {dataset} in {sample}, skipping")
            continue
        
        sample = ds.sample
        dataset = ds.dset
        
        print("Loading 3D peaks")
        cf_3d = ds.get_cf_3d_from_disk()
        cf_3d.parameters.loadparameters(ds.parfile)
        cf_3d.updateGeometry()
        cf_3d_path = f'{sample}_{dataset}_3d_peaks.flt'
        cf_3d.writefile(cf_3d_path)

        print("Filtering 3D peaks")
        cf_strong = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_frac, dsmax=cf_strong_dsmax, doplot=None, dstol=cf_strong_dstol)
        print(f"Got {cf_strong.nrows} strong peaks for indexing")
        cf_strong_path = f'{sample}_{dataset}_3d_peaks_strong.flt'
        cf_strong.writefile(cf_strong_path)

        cf_strong_allrings = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_allrings_frac, dsmax=cf_3d.ds.max(), doplot=None, dstol=cf_strong_allrings_dstol)
        print(f"Got {cf_strong_allrings.nrows} strong peaks for makemap")
        cf_strong_allrings_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt'
        cf_strong_allrings.writefile(cf_strong_allrings_path)
        
        grains, indexer = utils.do_index(cf=cf_strong,
                                        dstol=indexer.ds_tol,
                                        forgen=rings_for_gen,
                                        foridx=rings_for_scoring,
                                        hkl_tols=hkl_tols_seq,
                                        fracs=fracs,
                                        cosine_tol=cosine_tol,
                                        max_grains=max_grains
        )

        grains = [ImageD11.grain.grain(ubi, translation=np.array([0., 0., 0.])) for ubi in indexer.ubis]

        for i, g in enumerate(grains):
            g.gid = i

        tmp_ubi_path = f'{sample}_{dataset}_grains.ubi'
        tmp_map_path = f'{sample}_{dataset}_grains.map'

        new_flt_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt.new'  # flt file containing assignments from makemap
        unindexed_flt_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt.unindexed'  # remaining unassigned peaks from makemap

        ImageD11.grain.write_grain_file(tmp_ubi_path, grains)

        omegas_sorted = np.sort(ds.omega)[0]
        omega_slop = np.round(np.diff(omegas_sorted).mean(), 3)

        makemap_hkl_tol_seq = makemap_hkl_tol_seq

        for inc, makemap_tol in enumerate(makemap_hkl_tol_seq):
            print(f"Running makemap {inc+1}/{len(makemap_hkl_tol_seq)}")
            if inc == 0:  # ubi into map
                makemap_output = !makemap.py -p {ds.parfile} -u {tmp_ubi_path} -U {tmp_map_path} -f {cf_strong_allrings_path} -F {unindexed_flt_path} -s cubic -t {makemap_hkl_tol_seq[inc]} --omega_slop={omega_slop} --no_sort
            else:  # map into map
                makemap_output = !makemap.py -p {ds.parfile} -u {tmp_map_path} -U {tmp_map_path} -f {cf_strong_allrings_path} -F {unindexed_flt_path} -s cubic -t {makemap_hkl_tol_seq[inc]} --omega_slop={omega_slop} --no_sort

        grains2 = ImageD11.grain.read_grain_file(tmp_map_path)
        
        # remove grains with no peaks
        grains2 = [grain for grain in grains2 if "no peaks" not in grain.intensity_info]
        grains_filtered = [grain for grain in grains2 if float(grain.npks) > absolute_minpks]
        
        
        map_path = f'{sample}_{dataset}_grains_filtered.map'
        final_unindexed_flt_path = f'{sample}_{dataset}_3d_peaks.flt.unindexed'
        final_new_flt_path = f'{sample}_{dataset}_3d_peaks.flt.new'
        ImageD11.grain.write_grain_file(map_path, grains_filtered)
        makemap_output = !makemap.py -p {ds.parfile} -u {map_path} -U {map_path} -f {cf_3d_path} -F {final_unindexed_flt_path} -s {symmetry} -t {makemap_hkl_tol_seq[-1]} --omega_slop={omega_slop} --no_sort
        cf_3d = ImageD11.columnfile.columnfile(final_new_flt_path)
        ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile, name='peaks')
        grains_filtered = ImageD11.grain.read_grain_file(map_path)

        print("Saving grains")
        ds.save_grains_to_disk(grains_filtered)
        
        print("Cleaning up")
        for path in [
            cf_3d_path,
            cf_strong_path,
            cf_strong_allrings_path,
            tmp_ubi_path,
            tmp_map_path,
            new_flt_path,
            unindexed_flt_path,
            map_path,
            final_unindexed_flt_path,
            final_new_flt_path
        ]:
            if os.path.exists(path):
                os.remove(path)

print("Done!")